# Scikit-Learn SVM with NVFLARE


## Prepare data

In this section, we will download the data and split the data and save to the local disk

### Download data

In [1]:
from utils.prepare_data import download_data

The download data function will download one of the two datasets from Scikit-learn: Iris or Cancer
* the file will be save to the output directory 
* the file format will be CSV format with comma separated
* the file will be remove the header 
* default dataset is iris
* filename = dataset name


In [2]:
output_dir="/tmp/nvflare/sklearn/data"
download_data(output_dir)

Verify the file is downloaded


In [3]:
!ls {output_dir}

data_split.json  iris.csv  site-1  site-2  valid


### Split Data
* **Split Method**


Split the data into different datasets, one for each client. 
There are several split methods, we use test our algorithms in different scenarios. Here we just pick uniform split from the followns
* Uniform 
* linear
* Sqare
* Exponential



* **data store method**

There are two approaches to store the splited data 
* STORE DATA: 

similar to the real application, we split the data total into different directories (sites), and each client will ready one-site's data

```
   /tmp/nvflare/sklearn/data/site-1/iris.csv
   /tmp/nvflare/sklearn/data/site-2/iris.csv
   /tmp/nvflare/sklearn/data/valid/iris.csv
```

* STORE_INDEX: 

simulate the split, by assign data index range for each site, but the original file is not splited. The data loader is reading from the original file but only for the data within the index range
  For example: the index assignment for the data split is captured in a json file
 ``` 
  {
     "data_path" : "/tmp/nvflare/sklearn/data/iris.csv"
     "data_index" : {
         "site-1": {"start": 100, "end": 300},
         "site-2": {"start": 301, "end": 600},
     }
  }
 ```

Here we choose STORE_DATA approach

In [4]:
from utils.prepare_data_split import split_data, SplitMethod, StoreMethod

In [5]:
input_path = "/tmp/nvflare/sklearn/data/iris.csv"
output_dir = "/tmp/nvflare/sklearn/data"
site_num = 2
valid_frac = 0.3
split_method: SplitMethod = SplitMethod.UNIFORM
store_method = StoreMethod.STORE_DATA

In [6]:

split_data(input_path, output_dir, site_num, valid_frac, split_method=split_method)

In [7]:
!ls -l {output_dir}

total 20
-rw-rw-r-- 1 chester chester  316 Dec 17 10:52 data_split.json
-rw-rw-r-- 1 chester chester 3000 Dec 17 11:02 iris.csv
drwxrwxr-x 2 chester chester 4096 Dec 17 10:55 site-1
drwxrwxr-x 2 chester chester 4096 Dec 17 10:55 site-2
drwxrwxr-x 2 chester chester 4096 Dec 17 10:55 valid


In [8]:
! head -n 10 {output_dir}/site-1/iris.csv

0.0,4.8,3.0,1.4,0.3
0.0,5.1,3.8,1.6,0.2
0.0,4.6,3.2,1.4,0.2
0.0,5.3,3.7,1.5,0.2
0.0,5.0,3.3,1.4,0.2
1.0,7.0,3.2,4.7,1.4
1.0,6.4,3.2,4.5,1.5
1.0,6.9,3.1,4.9,1.5
1.0,5.5,2.3,4.0,1.3
1.0,6.5,2.8,4.6,1.5


## Config Jobs

ToDo 

## Running Job

### FL Simulator

In [10]:
! nvflare simulator -w /tmp/nvflare/ -n 2 -t 2 job_configs/sklearn_svm_base

2022-12-17 11:03:36,518 - SimulatorRunner - INFO - Create the Simulator Server.
2022-12-17 11:03:36,565 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 34991
2022-12-17 11:03:36,567 - SimulatorServer - INFO - starting insecure server at localhost:33347
2022-12-17 11:03:36,569 - SimulatorRunner - INFO - Deploy the Apps.
2022-12-17 11:03:36,571 - SimulatorRunner - INFO - Create the simulate clients.
2022-12-17 11:03:36,582 - ClientManager - INFO - Client: New client site-1@127.0.0.1 joined. Sent token: 70caa178-b98f-412e-897b-5e32eca3db41.  Total clients: 1
2022-12-17 11:03:36,583 - FederatedClient - INFO - Successfully registered client:site-1 for project simulator_server. Token:70caa178-b98f-412e-897b-5e32eca3db41 SSID:
2022-12-17 11:03:36,592 - ClientManager - INFO - Client: New client site-2@127.0.0.1 joined. Sent token: ea76b488-ab58-43f7-97b7-a6ee9405d38a.  Total clients: 2
2022-12-17 11:03:36,592 - FederatedClient - INFO - Successfully registered cli

Traceback (most recent call last):
  File "/home/chester/projects/NVFlare/nvflare/fuel/utils/json_scanner.py", line 96, in _do_scan
    node.processor.process_element(node)
  File "/home/chester/projects/NVFlare/nvflare/private/json_configer.py", line 122, in process_element
    self.process_config_element(self.config_ctx, node)
  File "/home/chester/projects/NVFlare/nvflare/private/fed/client/client_json_config.py", line 83, in process_config_element
    self.current_exe.executor = self.build_component(element)
  File "/home/chester/projects/NVFlare/nvflare/private/fed/client/client_json_config.py", line 87, in build_component
    t = super().build_component(config_dict)
  File "/home/chester/projects/NVFlare/nvflare/fuel/utils/component_builder.py", line 57, in build_component
    return instantiate_class(class_path, class_args)
  File "/home/chester/projects/NVFlare/nvflare/fuel/utils/class_utils.py", line 48, in instantiate_class
    c = get_class(class_path)
  File "/home/chester/

In [10]:
!ls -l  /tmp/nvflare/simulate_job/app_site-1

total 16
-rw-rw-r-- 1 chester chester    0 Dec 16 21:55 audit.log
drwxrwxr-x 2 chester chester 4096 Dec 16 21:55 config
drwxrwxr-x 3 chester chester 4096 Dec 16 21:55 custom
-rw-rw-r-- 1 chester chester   40 Dec 16 21:55 events.out.tfevents.1671256551.RTX.21091.0
-rw-rw-r-- 1 chester chester 1454 Dec 16 21:55 log.txt


In [2]:
!ls -l  /tmp/nvflare/sklearn/model

total 0
